In [1]:
! pip list

Package         Version   Editable project location
--------------- --------- ---------------------------------------------------------------------------------------------------------------
diabetes        0.1.0
diabetesoop     0.1.0     /Users/dyada/Desktop/BSE/Computing for Data Science/homeworks/hm6/homework-6-computing-data-science/diabetesoop
geographiclib   2.0
geopy           2.3.0
joblib          1.2.0
numpy           1.23.4
pandas          1.5.1
pip             22.3.1
python-dateutil 2.8.2
pytz            2022.6
scikit-learn    1.1.3
scipy           1.9.3
setuptools      65.5.1
six             1.16.0
sklearn         0.0.post1
threadpoolctl   3.1.0
wheel           0.37.1


In [2]:
import sys
package_path = '/Users/dyada/Desktop/BSE/Computing for Data Science/homeworks/hm6/homework-6-computing-data-science/diabetesoop/'
sys.path.append(package_path)

In [3]:
from diabetesoop.data.load import Data
from diabetesoop.processing.prep import DropNaNs, FillNaNs
from diabetesoop.processing.transformer import Dummifier, Binarizer
from diabetesoop.modeling.model import Model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [4]:
X_FEATURES = ['age', 'height', 'weight', 'aids', 'cirrhosis', 'hepatic_failure',
'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis', 'M/F']
Y_FEATURE = 'diabetes_mellitus'

#### a. Load the data.

In [5]:
data = Data()
X_y_train, X_y_test = data.from_csv('../data/sample_diabetes_mellitus_data.csv', Y_FEATURE)

print('X_y_train>>', X_y_train)
print('X_y_test>>', X_y_test)

X_y_train>>       encounter_id  hospital_id   age        bmi  elective_surgery  ethnicity  \
864         147116          118  67.0  36.095742                 0  Caucasian   
4211        271501          118  79.0        NaN                 0  Caucasian   
5980        252766          118  66.0        NaN                 0  Caucasian   
7504        222262          118  71.0  26.879192                 1  Caucasian   
9465        195030           89  43.0  32.681974                 0  Caucasian   
...            ...          ...   ...        ...               ...        ...   
6824        214796          118  61.0  31.207031                 1  Caucasian   
284         178354          118  78.0  24.234898                 0  Caucasian   
7058        154416          118  80.0  24.047337                 0  Caucasian   
5422        208812          118  65.0        NaN                 1      Asian   
1708        177376          118  55.0  42.578125                 0  Caucasian   

     gender  he

In [6]:
X_y_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8000 entries, 864 to 1708
Data columns (total 52 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   encounter_id                 8000 non-null   int64  
 1   hospital_id                  8000 non-null   int64  
 2   age                          7682 non-null   float64
 3   bmi                          6603 non-null   float64
 4   elective_surgery             8000 non-null   int64  
 5   ethnicity                    7807 non-null   object 
 6   gender                       7997 non-null   object 
 7   height                       7869 non-null   float64
 8   hospital_admit_source        7255 non-null   object 
 9   icu_admit_source             7981 non-null   object 
 10  icu_id                       8000 non-null   int64  
 11  icu_stay_type                8000 non-null   object 
 12  icu_type                     8000 non-null   object 
 13  pre_icu_los_days

In [7]:
X_y_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 5012 to 5575
Data columns (total 52 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   encounter_id                 2000 non-null   int64  
 1   hospital_id                  2000 non-null   int64  
 2   age                          1913 non-null   float64
 3   bmi                          1659 non-null   float64
 4   elective_surgery             2000 non-null   int64  
 5   ethnicity                    1951 non-null   object 
 6   gender                       2000 non-null   object 
 7   height                       1971 non-null   float64
 8   hospital_admit_source        1827 non-null   object 
 9   icu_admit_source             1998 non-null   object 
 10  icu_id                       2000 non-null   int64  
 11  icu_stay_type                2000 non-null   object 
 12  icu_type                     2000 non-null   object 
 13  pre_icu_los_day

## b. Remove those rows that contain NaN values in the columns: age, gender, ethnicity.

In [8]:
X_y_train = DropNaNs(X_y_train, ['age', 'gender', 'ethnicity']).drop()
X_y_test = DropNaNs(X_y_test, ['age', 'gender', 'ethnicity']).drop()

print('X_y_train>>', X_y_train.info())
print('X_y_test>>', X_y_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 864 to 1708
Data columns (total 52 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   encounter_id                 7503 non-null   int64  
 1   hospital_id                  7503 non-null   int64  
 2   age                          7503 non-null   float64
 3   bmi                          6195 non-null   float64
 4   elective_surgery             7503 non-null   int64  
 5   ethnicity                    7503 non-null   object 
 6   gender                       7503 non-null   object 
 7   height                       7401 non-null   float64
 8   hospital_admit_source        6816 non-null   object 
 9   icu_admit_source             7484 non-null   object 
 10  icu_id                       7503 non-null   int64  
 11  icu_stay_type                7503 non-null   object 
 12  icu_type                     7503 non-null   object 
 13  pre_icu_los_days

## c. Fill NaN with the mean value of the column in the columns: height, weight.

In [9]:
print('X_y_train nans before filling>>>\n', X_y_train.isna().sum().T[['height', 'weight']])
print('X_y_test nans before filling>>>\n', X_y_test.isna().sum().T[['height', 'weight']])

X_y_train = FillNaNs(X_y_train, ['height', 'weight']).fill()
X_y_test = FillNaNs(X_y_test, ['height', 'weight']).fill()

print('X_y_train nans after filling>>>\n', X_y_train.isna().sum().T[['height', 'weight']])
print('X_y_test nans after filling>>>\n', X_y_test.isna().sum().T[['height', 'weight']])

X_y_train nans before filling>>>
 height     102
weight    1256
dtype: int64
X_y_test nans before filling>>>
 height     25
weight    300
dtype: int64
X_y_train nans after filling>>>
 height    0
weight    0
dtype: int64
X_y_test nans after filling>>>
 height    0
weight    0
dtype: int64


## d. Transformations

In [10]:
# dummify ethnicity
X_y_train = Dummifier(X_y_train, ['ethnicity']).generate()
X_y_test = Dummifier(X_y_test, ['ethnicity']).generate()

print('X_y_train>>', X_y_train.info())
print('X_y_test>>', X_y_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 864 to 1708
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   encounter_id                 7503 non-null   int64  
 1   hospital_id                  7503 non-null   int64  
 2   age                          7503 non-null   float64
 3   bmi                          6195 non-null   float64
 4   elective_surgery             7503 non-null   int64  
 5   gender                       7503 non-null   object 
 6   height                       7503 non-null   float64
 7   hospital_admit_source        6816 non-null   object 
 8   icu_admit_source             7484 non-null   object 
 9   icu_id                       7503 non-null   int64  
 10  icu_stay_type                7503 non-null   object 
 11  icu_type                     7503 non-null   object 
 12  pre_icu_los_days             7503 non-null   float64
 13  readmission_stat

In [11]:
# binarize gender
X_y_train = Binarizer(X_y_train, 'gender', 'M/F').generate()
X_y_test = Binarizer(X_y_test, 'gender', 'M/F').generate()

print('X_y_train>>', X_y_train.info())
print('X_y_test>>', X_y_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 864 to 1708
Data columns (total 57 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   encounter_id                 7503 non-null   int64  
 1   hospital_id                  7503 non-null   int64  
 2   age                          7503 non-null   float64
 3   bmi                          6195 non-null   float64
 4   elective_surgery             7503 non-null   int64  
 5   height                       7503 non-null   float64
 6   hospital_admit_source        6816 non-null   object 
 7   icu_admit_source             7484 non-null   object 
 8   icu_id                       7503 non-null   int64  
 9   icu_stay_type                7503 non-null   object 
 10  icu_type                     7503 non-null   object 
 11  pre_icu_los_days             7503 non-null   float64
 12  readmission_status           7503 non-null   int64  
 13  weight          

### e. Model

In [12]:
# model_ = Model(feature_columns=X_FEATURES, target_column=Y_FEATURE, model=LogisticRegression, hyperparamaterers={'penalty': 'l2', 'solver': 'saga'})
model_ = Model(feature_columns=X_FEATURES, target_column=Y_FEATURE, model=RandomForestClassifier, hyperparamaterers={'n_estimators': 1000, 'criterion': 'gini'})

# f. Training & prediction

In [13]:
model_.train(X_y_train)
preds = model_.predict(X_y_test)
print('preds>>', preds)

Model has been fitted on training dataset!!
preds>> [[0.521   0.479  ]
 [0.993   0.007  ]
 [0.727   0.273  ]
 ...
 [0.75675 0.24325]
 [0.934   0.066  ]
 [0.965   0.035  ]]


# 7. roc_auc_score calculation

In [14]:
X_y_test['roc_auc_score'] = preds[:, 1]

print('AUC ROC score Train: ', roc_auc_score(X_y_test[Y_FEATURE], X_y_test['roc_auc_score']))

AUC ROC score Train:  0.62742663476874
